<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/gpt%203.5%20turbo%20-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install streamlit
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#vector_loader.py
#!pip install langchain-community < 0.0.27

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings


##new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
# to create a new file named vectorstore in your current directory.
def load_knowledgeBase():
        embeddings=OpenAIEmbeddings(api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh")
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
        return db

In [3]:
!pip install faiss-cpu
!pip install pypdf
import os
from concurrent.futures import ThreadPoolExecutor

# Specify the root directory where you want to search for PDF files
root_directory = "/content/sample_data/harry_porter"

# Set the batch size (number of files to process in each batch)
batch_size = 100

# Initialize an empty list to store loaded documents
docs = []

# Function to process a batch of PDF files
def process_pdf_batch(pdf_files):
    batch_docs = []
    for pdf_file_path in pdf_files:
        pdf_loader = PyPDFLoader(pdf_file_path)
        batch_docs.extend(pdf_loader.load())
    return batch_docs

# Get the list of PDF files to process
pdf_files_to_process = []
for root, dirs, files in os.walk(root_directory):
    pdf_files_to_process.extend([os.path.join(root, file) for file in files if file.lower().endswith(".pdf")])

# Create a ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    total_files = len(pdf_files_to_process)
    processed_files = 0

    # Iterate through the PDF files in batches
    for i in range(0, total_files, batch_size):
        batch = pdf_files_to_process[i:i+batch_size]
        batch_docs = list(executor.map(process_pdf_batch, [batch]))
        for batch_result in batch_docs:
            docs.extend(batch_result)
            processed_files += len(batch)
if __name__=="__main__":
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        #loader=PyPDFLoader("/content/sample_data/hp.pdf/harry-potter-book-collection-1-4.pdf")
        #docs=processed_files.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh"))
        vectorstore.save_local(DB_FAISS_PATH)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.2 MB/s eta 0:00:00


In [4]:
#Import Dependencies
from langchain.prompts import ChatPromptTemplate

def load_prompt():
        prompt = """ Answer as briefly as you can to the following question.You are required to give one-shot answer.
        context = {context}
        question = {question}
        if the answer is not in the pdf , answer "i donot know."
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt

In [5]:
#to load the OPENAI LLM
def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh")
        return llm

In [6]:
#Import Dependencies
!pip install faiss-cpu
import streamlit as sl
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

if __name__=='__main__':
        sl.header("welcome to the pdf bot")
        knowledgeBase=load_knowledgeBase()
        llm=load_llm()
        prompt=load_prompt()

        query=sl.text_input('Enter some text')


        if(query):
                #getting only the chunks that are similar to the query for llm to produce the output
                similar_embeddings=knowledgeBase.similarity_search(query)
                similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings(api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh"))

                #creating the chain for integrating llm,prompt,stroutputparser
                retriever = similar_embeddings.as_retriever()
                rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )

                response=rag_chain.invoke(query)
                sl.write(response)


2024-04-22 16:47:35.396 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [7]:
import streamlit as sl
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



def load_knowledgeBase():
        embeddings=OpenAIEmbeddings(api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh")
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization= True)
        return db

def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh")
        return llm


def load_prompt():
        prompt = """ Answer as briefly as you can to the following question. You are required to give a one shot answer
        context = {context}
        question = {question}
        if the answer is not in the pdf answer "i donot know what the hell you are asking about"
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt


def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

if __name__=='__main__':
        sl.header("welcome to the 📝PDF bot")
        sl.write("🤖 You can chat by Entering your queries ")
        knowledgeBase=load_knowledgeBase()
        llm=load_llm()
        prompt=load_prompt()

        query=sl.text_input('Enter some text')


        if(query):

                similar_embeddings=knowledgeBase.similarity_search(query)
                similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings(api_key="sk-proj-15uQaD8c6QjUCAdkajrpT3BlbkFJTlMleUzUOH4vcdhTingh"))

                retriever = similar_embeddings.as_retriever()
                rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )

                response=rag_chain.invoke(query)
                sl.write(response)


In [ ]:
!pip install streamlit


In [8]:
%%writefile app.py

Writing app.py


In [9]:
!  wget -q -O- ipv4.icanhazip.com

35.184.77.111


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.184.77.111:8501

npx: installed 22 in 3.377s
your url is: https://old-glasses-jog.loca.lt


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

if __name__=="__main__":
        DB_FAISS_PATH = 'vectorstore/db_faiss'
        loader=PyPDFLoader("/content/sample_data/harry-potter-book-collection-1-4.pdf")
        docs=loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(api_key=""))
        vectorstore.save_local(DB_FAISS_PATH)